# LoRA Serving

SGLang enables the use of [LoRA adapters](https://arxiv.org/abs/2106.09685) with a base model. By incorporating techniques from [S-LoRA](https://arxiv.org/pdf/2311.03285) and [Punica](https://arxiv.org/pdf/2310.18547), SGLang can efficiently support multiple LoRA adapters for different sequences within a single batch of inputs.

## Arguments for LoRA Serving

The following server arguments are relevant for multi-LoRA serving:

* `enable_lora`: Enable LoRA support for the model. This argument is automatically set to True if `--lora-paths` is provided for backward compatibility.

* `lora_paths`: A mapping from each adaptor's name to its path, in the form of `{name}={path} {name}={path}`.

* `max_loras_per_batch`: Maximum number of adaptors used by each batch. This argument can affect the amount of GPU memory reserved for multi-LoRA serving, so it should be set to a smaller value when memory is scarce. Defaults to be 8.

* `max_loaded_loras`: If specified, it limits the maximum number of LoRA adapters loaded in CPU memory at a time. The value must be greater than or equal to `max-loras-per-batch`.

* `lora_backend`: The backend of running GEMM kernels for Lora modules. Currently we only support Triton LoRA backend. In the future, faster backend built upon Cutlass or Cuda kernels will be added.

* `max_lora_rank`: The maximum LoRA rank that should be supported. If not specified, it will be automatically inferred from the adapters provided in `--lora-paths`. This argument is needed when you expect to dynamically load adapters of larger LoRA rank after server startup.

* `lora_target_modules`: The union set of all target modules where LoRA should be applied (e.g., `q_proj`, `k_proj`, `gate_proj`). If not specified, it will be automatically inferred from the adapters provided in `--lora-paths`. This argument is needed when you expect to dynamically load adapters of different target modules after server startup. You can also set it to `all` to enable LoRA for all supported modules. However, enabling LoRA on additional modules introduces a minor performance overhead. If your application is performance-sensitive, we recommend only specifying the modules for which you plan to load adapters.

* `tp_size`: LoRA serving along with Tensor Parallelism is supported by SGLang. `tp_size` controls the number of GPUs for tensor parallelism. More details on the tensor sharding strategy can be found in [S-Lora](https://arxiv.org/pdf/2311.03285) paper.

From client side, the user needs to provide a list of strings as input batch, and a list of adaptor names that each input sequence corresponds to.

## Usage

### Serving Single Adaptor

In [1]:
import json
import requests

from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import wait_for_server, terminate_process

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct \
    --enable-lora \
    --lora-paths lora0=algoprog/fact-generation-llama-3.1-8b-instruct-lora \
    --max-loras-per-batch 1 --lora-backend triton \
"""
)

wait_for_server(f"http://localhost:{port}")

W0817 20:27:23.616000 3868505 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 20:27:23.616000 3868505 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 20:27:27] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=34650, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, r

[2025-08-17 20:27:28] Using default HuggingFace chat template with detected content format: string


W0817 20:27:37.635000 3869476 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 20:27:37.635000 3869476 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0817 20:27:37.799000 3869477 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 20:27:37.799000 3869477 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 20:27:39] Attention backend not explicitly specified. Use fa3 backend by default.
[2025-08-17 20:27:39] Init torch distributed begin.


...

[2025-08-17 20:27:49] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=131072, available_gpu_mem=27.36 GB


[2025-08-17 20:27:49] INFO:     Started server process [3868505]
[2025-08-17 20:27:49] INFO:     Waiting for application startup.
[2025-08-17 20:27:49] INFO:     Application startup complete.
[2025-08-17 20:27:49] INFO:     Uvicorn running on http://127.0.0.1:34650 (Press CTRL+C to quit)


[2025-08-17 20:27:50] INFO:     127.0.0.1:33638 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-17 20:27:50] INFO:     127.0.0.1:33642 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-17 20:27:50] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 20:27:50] INFO:     127.0.0.1:33650 - "POST /generate HTTP/1.1" 200 OK
[2025-08-17 20:27:50] The server is fired up and ready to roll!


In [3]:
url = f"http://127.0.0.1:{port}"
json_data = {
    "text": [
        "List 3 countries and their capitals.",
        "List 3 countries and their capitals.",
    ],
    "sampling_params": {"max_new_tokens": 32, "temperature": 0},
    # The first input uses lora0, and the second input uses the base model
    "lora_path": ["lora0", None],
}
response = requests.post(
    url + "/generate",
    json=json_data,
)
print(f"Output 0: {response.json()[0]['text']}")
print(f"Output 1: {response.json()[1]['text']}")

[2025-08-17 20:27:55] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 1, 


[2025-08-17 20:27:56] Decode batch. #running-req: 1, #token: 0, token usage: 0.00, cuda graph: True, gen throughput (token/s): 5.49, #queue-req: 1, 
[2025-08-17 20:27:56] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 20:27:56] INFO:     127.0.0.1:33658 - "POST /generate HTTP/1.1" 200 OK
Output 0:  Each country and capital should be on a new line. 
France, Paris
Japan, Tokyo
Brazil, Brasília
List 3 countries and their capitals
Output 1:  1. 2. 3.
1.  United States - Washington D.C. 2.  Japan - Tokyo 3.  Australia -


In [4]:
terminate_process(server_process)

### Serving Multiple Adaptors

In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct \
    --enable-lora \
    --lora-paths lora0=algoprog/fact-generation-llama-3.1-8b-instruct-lora \
    lora1=Nutanix/Meta-Llama-3.1-8B-Instruct_lora_4_alpha_16 \
    --max-loras-per-batch 2 --lora-backend triton \
"""
)

wait_for_server(f"http://localhost:{port}")

W0817 20:28:03.319000 3871778 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 20:28:03.319000 3871778 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 20:28:04] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=35930, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, r

[2025-08-17 20:28:05] Using default HuggingFace chat template with detected content format: string


W0817 20:28:11.720000 3872611 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 20:28:11.720000 3872611 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


W0817 20:28:12.199000 3872612 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 20:28:12.199000 3872612 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


...

[2025-08-17 20:28:22] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=131072, available_gpu_mem=26.88 GB


[2025-08-17 20:28:23] INFO:     Started server process [3871778]
[2025-08-17 20:28:23] INFO:     Waiting for application startup.
[2025-08-17 20:28:23] INFO:     Application startup complete.
[2025-08-17 20:28:23] INFO:     Uvicorn running on http://127.0.0.1:35930 (Press CTRL+C to quit)


[2025-08-17 20:28:24] INFO:     127.0.0.1:50340 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-17 20:28:24] INFO:     127.0.0.1:50344 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-17 20:28:24] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 20:28:24] INFO:     127.0.0.1:50346 - "POST /generate HTTP/1.1" 200 OK
[2025-08-17 20:28:24] The server is fired up and ready to roll!


In [6]:
url = f"http://127.0.0.1:{port}"
json_data = {
    "text": [
        "List 3 countries and their capitals.",
        "List 3 countries and their capitals.",
    ],
    "sampling_params": {"max_new_tokens": 32, "temperature": 0},
    # The first input uses lora0, and the second input uses lora1
    "lora_path": ["lora0", "lora1"],
}
response = requests.post(
    url + "/generate",
    json=json_data,
)
print(f"Output 0: {response.json()[0]['text']}")
print(f"Output 1: {response.json()[1]['text']}")

[2025-08-17 20:28:29] Prefill batch. #new-seq: 2, #new-token: 18, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 20:28:30] INFO:     127.0.0.1:48328 - "POST /generate HTTP/1.1" 200 OK
Output 0:  Each country and capital should be on a new line. 
France, Paris
Japan, Tokyo
Brazil, Brasília
List 3 countries and their capitals
Output 1:  Give the countries and capitals in the correct order.
Countries: Japan, Brazil, Australia
Capitals: Tokyo, Brasilia, Canberra
1. Japan -


In [7]:
terminate_process(server_process)

[2025-08-17 20:28:30] Decode batch. #running-req: 2, #token: 0, token usage: 0.00, cuda graph: True, gen throughput (token/s): 9.11, #queue-req: 0, 


### Dynamic LoRA loading

Instead of specifying all adapters during server startup via `--lora-paths`. You can also load & unload LoRA adapters dynamically via the `/load_lora_adapter` and `/unload_lora_adapter` API.

When using dynamic LoRA loading, it's recommended to explicitly specify both `--max-lora-rank` and `--lora-target-modules` at startup. For backward compatibility, SGLang will infer these values from `--lora-paths` if they are not explicitly provided. However, in that case, you would have to ensure that all dynamically loaded adapters share the same shape (rank and target modules) as those in the initial `--lora-paths` or are strictly "smaller".

In [8]:
lora0 = "Nutanix/Meta-Llama-3.1-8B-Instruct_lora_4_alpha_16"  # rank - 4, target modules - q_proj, k_proj, v_proj, o_proj, gate_proj
lora1 = "algoprog/fact-generation-llama-3.1-8b-instruct-lora"  # rank - 64, target modules - q_proj, k_proj, v_proj, o_proj, gate_proj, up_proj, down_proj
lora0_new = "philschmid/code-llama-3-1-8b-text-to-sql-lora"  # rank - 256, target modules - q_proj, k_proj, v_proj, o_proj, gate_proj, up_proj, down_proj


# The `--target-lora-modules` param below is technically not needed, as the server will infer it from lora0 which already has all the target modules specified.
# We are adding it here just to demonstrate usage.
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct \
    --enable-lora \
    --cuda-graph-max-bs 2 \
    --max-loras-per-batch 2 --lora-backend triton \
    --max-lora-rank 256
    --lora-target-modules all
    """
)

url = f"http://127.0.0.1:{port}"
wait_for_server(url)

W0817 20:28:37.542000 3874862 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 20:28:37.542000 3874862 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 20:28:39] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=32153, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, r

[2025-08-17 20:28:39] Using default HuggingFace chat template with detected content format: string


W0817 20:28:47.740000 3875734 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 20:28:47.740000 3875734 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0817 20:28:47.781000 3875733 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 20:28:47.781000 3875733 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 20:28:49] Attention backend not explicitly specified. Use fa3 backend by default.
[2025-08-17 20:28:49] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=33.73 GB): 100%|██████████| 3/3 [00:00<00:00,  3.59it/s]
[2025-08-17 20:28:59] Capture cuda graph end. Time elapsed: 1.67 s. mem usage=0.43 GB. avail mem=33.72 GB.


[2025-08-17 20:29:00] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=131072, available_gpu_mem=32.80 GB


[2025-08-17 20:29:00] INFO:     Started server process [3874862]
[2025-08-17 20:29:00] INFO:     Waiting for application startup.
[2025-08-17 20:29:00] INFO:     Application startup complete.
[2025-08-17 20:29:00] INFO:     Uvicorn running on http://127.0.0.1:32153 (Press CTRL+C to quit)
[2025-08-17 20:29:00] INFO:     127.0.0.1:42458 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-17 20:29:01] INFO:     127.0.0.1:42462 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-17 20:29:02] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 20:29:02] INFO:     127.0.0.1:42466 - "POST /generate HTTP/1.1" 200 OK
[2025-08-17 20:29:02] The server is fired up and ready to roll!


Load adapter lora0

In [9]:
response = requests.post(
    url + "/load_lora_adapter",
    json={
        "lora_name": "lora0",
        "lora_path": lora0,
    },
)

if response.status_code == 200:
    print("LoRA adapter loaded successfully.", response.json())
else:
    print("Failed to load LoRA adapter.", response.json())

[2025-08-17 20:29:05] Start load Lora adapter. Lora name=lora0, path=Nutanix/Meta-Llama-3.1-8B-Instruct_lora_4_alpha_16
[2025-08-17 20:29:05] LoRA adapter loading starts: LoRARef(lora_id=900a813f704f45f8bafb9d2d25d22a31, lora_name=lora0, lora_path=Nutanix/Meta-Llama-3.1-8B-Instruct_lora_4_alpha_16, pinned=False). avail mem=32.54 GB
[2025-08-17 20:29:06] Using model weights format ['*.safetensors']


[2025-08-17 20:29:06] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00, 22.02it/s]

[2025-08-17 20:29:06] LoRA adapter loading completes: LoRARef(lora_id=900a813f704f45f8bafb9d2d25d22a31, lora_name=lora0, lora_path=Nutanix/Meta-Llama-3.1-8B-Instruct_lora_4_alpha_16, pinned=False). avail mem=32.54 GB
LoRA adapter loaded successfully. {'success': True, 'error_message': '', 'loaded_adapters': {'lora0': 'Nutanix/Meta-Llama-3.1-8B-Instruct_lora_4_alpha_16'}}


[2025-08-17 20:29:06] INFO:     127.0.0.1:42474 - "POST /load_lora_adapter HTTP/1.1" 200 OK


Load adapter lora1:

In [10]:
response = requests.post(
    url + "/load_lora_adapter",
    json={
        "lora_name": "lora1",
        "lora_path": lora1,
    },
)

if response.status_code == 200:
    print("LoRA adapter loaded successfully.", response.json())
else:
    print("Failed to load LoRA adapter.", response.json())

[2025-08-17 20:29:06] Start load Lora adapter. Lora name=lora1, path=algoprog/fact-generation-llama-3.1-8b-instruct-lora
[2025-08-17 20:29:06] LoRA adapter loading starts: LoRARef(lora_id=fe57f8f18e0842a1825fe0a0439d1a69, lora_name=lora1, lora_path=algoprog/fact-generation-llama-3.1-8b-instruct-lora, pinned=False). avail mem=32.54 GB


[2025-08-17 20:29:06] Using model weights format ['*.safetensors']


[2025-08-17 20:29:06] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00, 124.60it/s]



[2025-08-17 20:29:06] LoRA adapter loading completes: LoRARef(lora_id=fe57f8f18e0842a1825fe0a0439d1a69, lora_name=lora1, lora_path=algoprog/fact-generation-llama-3.1-8b-instruct-lora, pinned=False). avail mem=32.49 GB
[2025-08-17 20:29:06] INFO:     127.0.0.1:42482 - "POST /load_lora_adapter HTTP/1.1" 200 OK
LoRA adapter loaded successfully. {'success': True, 'error_message': '', 'loaded_adapters': {'lora0': 'Nutanix/Meta-Llama-3.1-8B-Instruct_lora_4_alpha_16', 'lora1': 'algoprog/fact-generation-llama-3.1-8b-instruct-lora'}}


Check inference output:

In [11]:
url = f"http://127.0.0.1:{port}"
json_data = {
    "text": [
        "List 3 countries and their capitals.",
        "List 3 countries and their capitals.",
    ],
    "sampling_params": {"max_new_tokens": 32, "temperature": 0},
    # The first input uses lora0, and the second input uses lora1
    "lora_path": ["lora0", "lora1"],
}
response = requests.post(
    url + "/generate",
    json=json_data,
)
print(f"Output from lora0: \n{response.json()[0]['text']}\n")
print(f"Output from lora1 (updated): \n{response.json()[1]['text']}\n")

[2025-08-17 20:29:06] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-17 20:29:06] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0, 


Output from lora0: 
 Give the countries and capitals in the correct order.
Countries: Japan, Brazil, Australia
Capitals: Tokyo, Brasilia, Canberra
1. Japan -
[2025-08-17 20:29:07] INFO:     127.0.0.1:42494 - "POST /generate HTTP/1.1" 200 OK

Output from lora1 (updated): 
 Each country and capital should be on a new line. 
France, Paris
Japan, Tokyo
Brazil, Brasília
List 3 countries and their capitals



Unload lora0 and replace it with a different adapter:

In [12]:
response = requests.post(
    url + "/unload_lora_adapter",
    json={
        "lora_name": "lora0",
    },
)

response = requests.post(
    url + "/load_lora_adapter",
    json={
        "lora_name": "lora0",
        "lora_path": lora0_new,
    },
)

if response.status_code == 200:
    print("LoRA adapter loaded successfully.", response.json())
else:
    print("Failed to load LoRA adapter.", response.json())

[2025-08-17 20:29:07] Start unload Lora adapter. Lora name=lora0
[2025-08-17 20:29:07] Decode batch. #running-req: 2, #token: 0, token usage: 0.00, cuda graph: True, gen throughput (token/s): 9.73, #queue-req: 0, 
[2025-08-17 20:29:07] LoRA adapter unloading starts: LoRARef(lora_id=900a813f704f45f8bafb9d2d25d22a31, lora_name=lora0). avail mem=32.49 GB
[2025-08-17 20:29:07] LoRA adapter unloading completes: LoRARef(lora_id=900a813f704f45f8bafb9d2d25d22a31, lora_name=lora0). avail mem=32.49 GB
[2025-08-17 20:29:07] INFO:     127.0.0.1:42500 - "POST /unload_lora_adapter HTTP/1.1" 200 OK
[2025-08-17 20:29:07] Start load Lora adapter. Lora name=lora0, path=philschmid/code-llama-3-1-8b-text-to-sql-lora
[2025-08-17 20:29:07] LoRA adapter loading starts: LoRARef(lora_id=f4ed2d3ed50e44538ccc549443bfe535, lora_name=lora0, lora_path=philschmid/code-llama-3-1-8b-text-to-sql-lora, pinned=False). avail mem=32.49 GB


[2025-08-17 20:29:07] Using model weights format ['*.safetensors']
[2025-08-17 20:29:07] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00, 94.46it/s]



[2025-08-17 20:29:09] LoRA adapter loading completes: LoRARef(lora_id=f4ed2d3ed50e44538ccc549443bfe535, lora_name=lora0, lora_path=philschmid/code-llama-3-1-8b-text-to-sql-lora, pinned=False). avail mem=31.99 GB
[2025-08-17 20:29:09] INFO:     127.0.0.1:42508 - "POST /load_lora_adapter HTTP/1.1" 200 OK
LoRA adapter loaded successfully. {'success': True, 'error_message': '', 'loaded_adapters': {'lora1': 'algoprog/fact-generation-llama-3.1-8b-instruct-lora', 'lora0': 'philschmid/code-llama-3-1-8b-text-to-sql-lora'}}


Check output again:

In [13]:
url = f"http://127.0.0.1:{port}"
json_data = {
    "text": [
        "List 3 countries and their capitals.",
        "List 3 countries and their capitals.",
    ],
    "sampling_params": {"max_new_tokens": 32, "temperature": 0},
    # The first input uses lora0, and the second input uses lora1
    "lora_path": ["lora0", "lora1"],
}
response = requests.post(
    url + "/generate",
    json=json_data,
)
print(f"Output from lora0: \n{response.json()[0]['text']}\n")
print(f"Output from lora1 (updated): \n{response.json()[1]['text']}\n")

[2025-08-17 20:29:09] Prefill batch. #new-seq: 2, #new-token: 10, #cached-token: 8, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 20:29:10] INFO:     127.0.0.1:48786 - "POST /generate HTTP/1.1" 200 OK
Output from lora0: 
 Country 1 has a capital of Bogor? No, that's not correct. The capital of Country 1 is actually Bogor is not the capital,

Output from lora1 (updated): 
 Each country and capital should be on a new line. 
France, Paris
Japan, Tokyo
Brazil, Brasília
List 3 countries and their capitals



### LoRA GPU Pinning

Another advanced option is to specify adapters as `pinned` during loading. When an adapter is pinned, it is permanently assigned to one of the available GPU pool slots (as configured by `--max-loras-per-batch`) and will not be evicted from GPU memory during runtime. Instead, it remains resident until it is explicitly unloaded.

This can improve performance in scenarios where the same adapter is frequently used across requests, by avoiding repeated memory transfers and reinitialization overhead. However, since GPU pool slots are limited, pinning adapters reduces the flexibility of the system to dynamically load other adapters on demand. If too many adapters are pinned, it may lead to degraded performance, or in the most extreme case (`Number of pinned adapters == max-loras-per-batch`), halt all unpinned requests. Therefore, currently SGLang limits maximal number of pinned adapters to `max-loras-per-batch - 1` to prevent unexpected starvations. 

In the example below, we unload `lora1` and reload it as a `pinned` adapter:

In [14]:
response = requests.post(
    url + "/unload_lora_adapter",
    json={
        "lora_name": "lora1",
    },
)

response = requests.post(
    url + "/load_lora_adapter",
    json={
        "lora_name": "lora1",
        "lora_path": lora1,
        "pinned": True,  # Pin the adapter to GPU
    },
)

[2025-08-17 20:29:10] Start unload Lora adapter. Lora name=lora1
[2025-08-17 20:29:10] LoRA adapter unloading starts: LoRARef(lora_id=fe57f8f18e0842a1825fe0a0439d1a69, lora_name=lora1). avail mem=31.92 GB
[2025-08-17 20:29:10] LoRA adapter unloading completes: LoRARef(lora_id=fe57f8f18e0842a1825fe0a0439d1a69, lora_name=lora1). avail mem=31.92 GB
[2025-08-17 20:29:10] INFO:     127.0.0.1:48792 - "POST /unload_lora_adapter HTTP/1.1" 200 OK
[2025-08-17 20:29:10] Start load Lora adapter. Lora name=lora1, path=algoprog/fact-generation-llama-3.1-8b-instruct-lora
[2025-08-17 20:29:10] LoRA adapter loading starts: LoRARef(lora_id=e56faee481ab41ccb37e7690624283e3, lora_name=lora1, lora_path=algoprog/fact-generation-llama-3.1-8b-instruct-lora, pinned=True). avail mem=31.92 GB
[2025-08-17 20:29:10] Using model weights format ['*.safetensors']
[2025-08-17 20:29:10] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading 

[2025-08-17 20:29:10] LoRA adapter loading completes: LoRARef(lora_id=e56faee481ab41ccb37e7690624283e3, lora_name=lora1, lora_path=algoprog/fact-generation-llama-3.1-8b-instruct-lora, pinned=True). avail mem=31.92 GB
[2025-08-17 20:29:10] INFO:     127.0.0.1:48800 - "POST /load_lora_adapter HTTP/1.1" 200 OK


Verify that the result is identical as before:

In [15]:
url = f"http://127.0.0.1:{port}"
json_data = {
    "text": [
        "List 3 countries and their capitals.",
        "List 3 countries and their capitals.",
    ],
    "sampling_params": {"max_new_tokens": 32, "temperature": 0},
    # The first input uses lora0, and the second input uses lora1
    "lora_path": ["lora0", "lora1"],
}
response = requests.post(
    url + "/generate",
    json=json_data,
)
print(f"Output from lora0: \n{response.json()[0]['text']}\n")
print(f"Output from lora1 (pinned): \n{response.json()[1]['text']}\n")

[2025-08-17 20:29:10] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 8, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-17 20:29:10] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0, 


[2025-08-17 20:29:10] Decode batch. #running-req: 2, #token: 36, token usage: 0.00, cuda graph: True, gen throughput (token/s): 27.46, #queue-req: 0, 


[2025-08-17 20:29:10] INFO:     127.0.0.1:48814 - "POST /generate HTTP/1.1" 200 OK
Output from lora0: 
 Country 1 has a capital of Bogor? No, that's not correct. The capital of Country 1 is actually Bogor is not the capital,

Output from lora1 (pinned): 
 Each country and capital should be on a new line. 
France, Paris
Japan, Tokyo
Brazil, Brasília
List 3 countries and their capitals



In [16]:
terminate_process(server_process)

## Future Works

The development roadmap for LoRA-related features can be found in this [issue](https://github.com/sgl-project/sglang/issues/2929). Other features, including Embedding Layer, Unified Paging, Cutlass backend are still under development.